In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted
import unicodedata

### Variabili Globali

In [3]:
data_folder_in = 'drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/'
data_folder_out = 'drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/'

file_in = 'dataset_divina_*.csv'

file_out = 'dataset_dantesco.csv'

In [4]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    '''
    Preprocessing dei testi di input, impostando tutti i caratteri
    minuscoli, aggiungendo uno spazio prima di ogni punto e sostituendo
    qualsiasi carattere con uno spazio se non è compreso nel seguente elenco:
    (a-z, A-Z, ".", "?", "!", ",")
    '''
    w = unicode_to_ascii(w.lower().strip())

    w = w.replace('..', '')
    
    return w

In [5]:
def verifica_correttezza(df):
  original = df['Original']
  translate = df['Translate']

  print(f'Esempi nel Dataset                 : {len(df)}')
  print(f'Frase più corta in Original        : {min(original, key = len)}')
  print(f'Frase più corta in Translate       : {min(translate, key = len)}')
  print(f'Frase più lunga in Original        : {max(original, key = len)}')
  print(f'Frase più lunga in Translate       : {max(translate, key = len)}')
  print('---------------------------------------------------------------------------------------')

In [6]:
df = pd.DataFrame(columns=['Original', 'Translate'])

filenamepath = os.path.abspath(os.path.join(data_folder_in, file_in))
print(glob.glob(filenamepath))
for filename in natsorted(glob.glob(filenamepath)):
  df_filename = pd.read_csv(filename, 
                            usecols=[1,2],
                            skiprows=[0],
                            names=['Original', 'Translate'])
  
  print(filename)
  # Preprocessing dei dati di Input
  df_filename['Original'] = df_filename['Original'].apply(lambda x : preprocess_sentence(x))
  
  # Preprocessing dei dati Tradotti  
  df_filename['Translate'] = df_filename['Translate'].apply(lambda x : preprocess_sentence(x)).tolist()

  verifica_correttezza(df_filename)

  df = pd.concat([df, df_filename[['Original', 'Translate']]])

['/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_inferno_4.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_inferno_5.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_inferno_2.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_inferno_1.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_purgatorio_1.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_purgatorio_4.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_purgatorio_2.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_purgatorio_3.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/dataset_divina_commedia_paradiso_2.csv', '/content/drive/MyDrive/Traduzione_Dantesca/Dante_Transformer/datase

In [7]:
df

,Original,Translate
0,nel mezzo del cammin di nostra vita,a meta del cammino della nostra vita
1,"mi ritrovai per una selva oscura,","mi sono ritrovato in una foresta buia,"
2,che la diritta via era smarrita.,per il percorso semplice era stato perso.
3,ahi quanto a dir qual era e cosa dura,ah me! quanto e difficile una cosa da dire
4,esta selva selvaggia e aspra e forte,"cos'era questa foresta selvaggia, ruvida e sev..."
...,...,...
4477,tosto che e per segno fuor dischiusa;,"colui che gli si mostro per bocca e per bocca,"
4478,"cosi, poi che da essa preso fui,","cosi, dopo avermi preso per il braccio,"
4479,"la bella donna mossesi, e a stazio",la bella donna e ha detto femminile mite
4480,donnescamente disse: “vien con lui”.,"a stazio: ""anche tu devi venire con lui."""


In [8]:
df = df.dropna()
df = df.reset_index(drop=True)

filenamepath_out = os.path.abspath(os.path.join(data_folder_out, file_out))
df.to_csv(filenamepath_out, index=False, encoding='utf-8')